In [1]:
%%capture --no-stderr
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import warnings
import nest_asyncio
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")
nest_asyncio.apply()
_ = load_dotenv(find_dotenv())

# 1. Building our first multi-agent system with LlamaIndex Workflows!

## a. Building our RAG workflow

In [3]:
from phoenix.otel import register
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

## Log traces using Llama Trace
tracer_provider = register(
  project_name="llama-trace-v1",
  endpoint="https://app.phoenix.arize.com/v1/traces"
)
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: llama-trace-v1
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



### i. Building our RAG Service

In [4]:
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2021/q4/_10-K-2021-(As-Filed).pdf" -O apple_2021_10k.pdf

--2024-11-09 13:49:00--  https://s2.q4cdn.com/470004039/files/doc_financials/2021/q4/_10-K-2021-(As-Filed).pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 139.99.123.118
Connecting to s2.q4cdn.com (s2.q4cdn.com)|139.99.123.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 789896 (771K) [application/pdf]
Saving to: ‘apple_2021_10k.pdf’

apple_2021_10k.pdf  100%[===================>] 771.38K  --.-KB/s    in 0.06s   

2024-11-09 13:49:00 (13.7 MB/s) - ‘apple_2021_10k.pdf’ saved [789896/789896]



### ii. Ingest document

In [4]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

llm = OpenAI("gpt-4o-mini")
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [5]:
documents = SimpleDirectoryReader(
    input_files=["./apple_2021_10k.pdf"]
).load_data()

#### Let's ingest the Apple 10K pdf file into a hybrid search enabled vector database - Qdrant!
Set up Qdrant vector database

In [ ]:
# !docker run -p 6333:6333 -p 6334:6334 \
#     -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
#     qdrant/qdrant

In [14]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient

client = QdrantClient(host="localhost", port=6333)
aclient = AsyncQdrantClient(host="localhost", port=6333)

# delete collection if it exists
if client.collection_exists("Apple_10K"):
    client.delete_collection("Apple_10K")

vector_store = QdrantVectorStore(
    "Apple_10K",
    client = client,
    aclient = aclient,
    enable_hybrid = True,
    fastembed_sparse_model="Qdrant/bm42-all-minilm-l6-v2-attentions",
)

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:6333/collections/Apple_10K "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K/exists "HTTP/1.1 200 OK"


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K/exists "HTTP/1.1 200 OK"


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [15]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [16]:
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model = embed_model
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/Apple_10K "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/Apple_10K/index?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/Apple_10K/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/Apple_10K/points?wait=true "HTTP/1.1 200 OK"


#### Load from storage context

In [6]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient

client = QdrantClient(host="localhost", port=6333)
aclient = AsyncQdrantClient(host="localhost", port=6333)


vector_store = QdrantVectorStore(
    "Apple_10K",
    client = client,
    aclient = aclient,
    enable_hybrid = True,
    fastembed_sparse_model="Qdrant/bm42-all-minilm-l6-v2-attentions",
)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K "HTTP/1.1 200 OK"


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Apple_10K "HTTP/1.1 200 OK"


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
query_engine = index.as_query_engine(
    similarity_top_k=2, sparse_top_k=12, vector_store_query_mode="hybrid", llm = llm
)

In [8]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="apple_10k",
            description=(
                "Provides information about Apple financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            )
        )
    )
]

In [54]:
from llama_index.core.agent import FunctionCallingAgent

apple_agent = FunctionCallingAgent.from_tools(
    tools=tools,
    llm = llm,
    verbose = True,
)

## b. Add our DSPy-LlamaIndex HyDE and Seek tool
> Using our Gemini LLM!

#### i. Get data

In [17]:
from llama_index.core import Document
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.readers.web import SimpleWebPageReader
import dspy
from dspy import (
    Signature,
    InputField,
    OutputField,
    Module,
    Predict,
    Prediction
)
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import answer_exact_match, answer_passage_match
from dspy import Example

import requests

In [11]:
llm2 = Gemini(model="models/gemini-1.5-flash")
embed_model2 = GeminiEmbedding()

In [12]:
class CustomWebPageReader(SimpleWebPageReader):
    """
    Many websites, including Investopedia, require headers like User-Agent to be set in the request to return the correct content.
    To fix this, we'll modify the load_data method in the SimpleWebPageReader class to include appropriate headers.
    """
    
    def load_data(self, urls):
        """Edit the headers portion in the load_data method to be able to read .asp files"""
        
        if not isinstance(urls, list):
            raise ValueError("urls must be a list of strings.")
        documents = []
        
        ## This is the edit
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        for url in urls:
            response = requests.get(url, headers=headers).text
            if self.html_to_text:
                import html2text
                response = html2text.html2text(response)

            metadata = None
            if self._metadata_fn is not None:
                metadata = self._metadata_fn(url)

            documents.append(Document(text=response, id_=url, metadata=metadata or {}))

        return documents

In [13]:
links = [
    "https://www.investopedia.com/terms/s/stockmarket.asp",
    "https://www.investopedia.com/ask/answers/difference-between-options-and-futures/",
    "https://www.investopedia.com/financial-edge/0411/5-essential-things-you-need-to-know-about-every-stock-you-buy.aspx",
    "https://www.investopedia.com/articles/fundamental/04/063004.asp",
    "https://www.investopedia.com/terms/t/technicalanalysis.asp",
    "https://www.investopedia.com/terms/i/ichimoku-cloud.asp",
    "https://www.investopedia.com/terms/a/aroon.asp",  
    "https://www.investopedia.com/terms/b/bollingerbands.asp",
    "https://www.investopedia.com/articles/forex/05/macddiverge.asp",
    "https://www.investopedia.com/terms/a/accumulationdistribution.asp",
    "https://www.investopedia.com/terms/s/stochasticoscillator.asp",
    "https://www.investopedia.com/terms/s/stochrsi.asp",
    "https://www.investopedia.com/terms/p/price-earningsratio.asp",
    "https://www.investopedia.com/terms/p/price-to-bookratio.asp",
    "https://www.investopedia.com/terms/p/price-to-salesratio.asp",
    "https://www.investopedia.com/terms/q/quickratio.asp"
]

In [18]:
docs = CustomWebPageReader(
    html_to_text=True
).load_data(urls=links)

In [27]:
investopedia_index = VectorStoreIndex.from_documents(
    docs,
    embed_model=embed_model2,
)

In [32]:
investopedia_index.storage_context.persist(persist_dir="../investopedia_storage")

#### ii. Load from storage

In [34]:
investopedia_storage_context = StorageContext.from_defaults(persist_dir="../investopedia_storage")
investopedia_index = load_index_from_storage(investopedia_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.


#### iv. DSPy Finetuning

In [40]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform

In [41]:
hyde = HyDEQueryTransform(include_original=True, llm = llm2)

In [36]:
retriever = investopedia_index.as_retriever(embed_model=embed_model2)

In [39]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="May contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="Often between 1-5 sentences.")

In [42]:
class HydeRAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retriever = retriever
        self.generate_answer = dspy.Predict(GenerateAnswer)
    
    def get_context(self, question):
        hyded_query = hyde.run(question)
        hyde_nodes = retriever.retrieve(hyded_query.custom_embedding_strs[0]) #retrieve nodes for hypothetical document
        nodes = retriever.retrieve(hyded_query.custom_embedding_strs[1]) #retrieve nodes for original question
        return "\n\n".join([n.get_content() for n in hyde_nodes]) + "\n\n".join([n.get_content() for n in nodes])
    
    def forward(self, question):
        context = self.get_context(question)
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer = prediction.answer)

#### v. Test HyDE Rag

In [46]:
lm = dspy.Google(model="gemini-1.5-flash")
dspy.settings.configure(lm=lm)

In [47]:
engine = HydeRAG()
question = "What is a dividend?"
pred = engine(question)

 		You are using the client Google, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


#### vi. Finetuning!

In [43]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator

evaluator = SemanticSimilarityEvaluator(
    similarity_threshold=0.5, 
    embed_model=embed_model2
)

def validate_context_and_answer(example, pred, trace=None):
    result = evaluator.evaluate(
        response = pred.answer,
        reference = example.answer
    )
    return result.passing

In [44]:
train_examples = [
        Example(
            question = "What is the difference between an option and a future?",
            answer = """An option gives the buyer the right, but not the obligation, to buy (or sell) an asset at a specific price at any time during the life of the contract.
            A futures contract obligates the buyer to purchase a specific asset, and the seller to sell and deliver that asset, at a specific future date
            """
        ),
        Example(
            question = "What are the similarities between an option and a future?",
            answer = """Futures and options positions may be traded and closed ahead of expiration, but the parties to the futures contracts for commodities are typically obligated to make and accept deliveries on the settlement date."""
        ),
        Example(
            question = "What is an option?",
            answer = "Options are financial derivatives. An options contract gives an investor the right to buy or sell the underlying instrument at a specific price while the contract is in effect. Investors may choose not to exercise their options. Option holders do not own the underlying shares or enjoy shareholder rights unless they exercise an option to buy stock."
        ),
        Example(
            question = "What are the different options?",
            answer = "There are only two kinds of options: Call options and put options. A call option confers the right to buy a stock at the strike price before the agreement expires. A put option gives the holder the right to sell a stock at a specific price."
        ),
        Example(
            question="What's P/E ratio?",
            answer = """ This ratio is used to measure a company's current share price relative to its per-share earnings. The company can be compared to other, similar corporations so that analysts and investors can determine its relative value. So if a company has a P/E ratio of 20, this means investors are willing to pay $20 for every $1 per earnings. That might seem expensive but not if the company is growing fast. The P/E can be found by comparing the current market price to the cumulative earnings of the last four quarters."""
        ),
        Example(
            question="What's a dividend?",
            answer = """Dividends are like interest in a savings account—you get paid regardless of the stock price. Dividends are distributions made by a company to its shareholders as a reward from its profits. The amount of the dividend is decided by its board of directors and are generally issued in cash, though it isn't uncommon for some companies to issue dividends in the form of stock shares."""
        ),
        Example(
            question="What's a balance sheet?",
            answer=" A balance sheet is a financial statement that reports a company's assets, liabilities and shareholder equity at a specific point in time"
        ),
        Example(
            question="What's a current ratio?",
            answer = "It's the total current assets divided by total current liabilities, commonly used by analysts to assess the ability of a company to meet its short-term obligations"
        ),
        Example(
            question = "What are stocks?",
            answer = "When you buy a stock or a share, you're getting a piece of that company. Owning shares gives you the right to part of the company's profits, often paid as dividends, and sometimes the right to vote on company matters"
        ),
        Example(
            question = "What are REITs?",
            answer="Real estate investment trusts (REITs) are companies that own, manage, or finance real estate. Investors can buy shares in them, and they legally must provide 90% of their profits as dividends each year."
        ),
        Example(
            question = "What are brokers?",
            answer = "Brokers in the stock market play the same role as in insurance and elsewhere, acting as a go-between for investors and the securities markets. They are licensed organizations that buy and sell stocks and other securities for individual and institutional clients."
        ),
        Example(
            question = "What is technical analysis?",
            answer = "Technical analysis is used to scrutinize the ways supply and demand for a security affect changes in price, volume, and implied volatility. It assumes that past trading activity and price changes of a security can be valuable indicators of the security's future price movements when paired with appropriate investing or trading rules."
        ),
        Example(
            question="What is the difference between fundamental and technical analysis?",
            answer = "Fundamental analysis is a method of evaluating securities by attempting to measure the intrinsic value of a stock. Fundamental analysts study everything from the overall economy and industry conditions to the financial condition and management of companies. Technical analysis differs from fundamental analysis in that the stock's price and volume are the only inputs. The core assumption is that all publicly known fundamentals have factored into price; thus, there is no need to pay close attention to them. Technical analysts do not attempt to measure a security's intrinsic value, but instead, use stock charts to identify patterns and trends that suggest how a stock's price will move in the future."
        )
    ]
train_examples = [t.with_inputs("question") for t in train_examples]

In [48]:
teleprompter = BootstrapFewShot(
    max_labeled_demos=0,
    metric=validate_context_and_answer
)
compiled_dspy_qp = teleprompter.compile(engine, trainset=train_examples)

  0%|          | 0/13 [00:00<?, ?it/s]ERROR:dspy.teleprompt.bootstrap:2024-11-09T14:42:47.946296Z [error    ] Failed to run or to evaluate example Example({'question': 'What is the difference between an option and a future?', 'answer': 'An option gives the buyer the right, but not the obligation, to buy (or sell) an asset at a specific price at any time during the life of the contract.\n            A futures contract obligates the buyer to purchase a specific asset, and the seller to sell and deliver that asset, at a specific future date\n            '}) (input_keys={'question'}) with <function validate_context_and_answer at 0x3e35ed300> due to index: 0
finish_reason: RECITATION
. [dspy.teleprompt.bootstrap] filename=bootstrap.py lineno=211
 38%|███▊      | 5/13 [00:41<01:06,  8.27s/it]

Bootstrapped 4 full traces after 6 examples in round 0.


In [49]:
compiled_dspy_qp.save("../compiled.json")

[('generate_answer', Predict(GenerateAnswer(context, question -> answer
    instructions='Answer questions with short factoid answers.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'May contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'Often between 1-5 sentences.', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)))]


#### vii. Test loading saved pipeline

In [50]:
## Loading a saved pipeline

pipeline = HydeRAG()
pipeline.load("../compiled.json")

question = "What is the difference between bollinger bands and the ichimoku cloud?"
pred = pipeline(question)
print(f"Question: {question}")
print(f"Predicted Answer: {pred.answer}")

Question: What is the difference between bollinger bands and the ichimoku cloud?
Predicted Answer: The Ichimoku Cloud uses averages based on highs and lows over a period, while Bollinger Bands use a simple moving average of closing prices.


## Building our LlamaIndex Multi-Agent System using Workflow!

In [68]:
from llama_index.core.workflow import (
    Workflow,
    step,
    Context,
    Event,
    StartEvent,
    StopEvent
)
from llama_index.core.base.llms.types import ChatMessage
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import Literal, List, Dict, Optional, Any, Union, Annotated

In [61]:
class AppleEvent(Event):
    query: str
    history: List[ChatMessage]

class InvestopediaEvent(Event):
    query: str #Stuff chat history into the query

class CompileEvent(Event):
    query: str
    response: str

In [65]:
class Reroute(BaseModel):
    reroute: Literal[True, False] = Field(..., description="If the answer is satisfactory, return True. Else return False for rerouting")
    improvements: str = Field(..., description="Suggestions to improve the response.")

In [ ]:
class StockSME(Workflow):
    def __init__(self, history = List[str], manager_llm = llm):
        
        self.llm = llm
        
        ## Initialize the "manager"
        self.manager_llm = SimpleChatEngine.from_defaults(llm=manager_llm)
        
        self.choices = [
            ToolMetadata(
                name = "Apple_Agent",
                description = """Choose this option to answer any questions related to Apple's financial performance in 2021
                including risks, opportunities, and financial statements."""
            ),
            ToolMetadata(
                name = "Definitions_Agent",
                description = "Choose this option to answer any questions related to investment definitions such as Ichimoku Cloud"
            )
        ] 
        self.router = LLMSingleSelector.from_defaults(llm=self.llm)
        self.reroute_prompt_template = PromptTemplate(
            """Given the query: {query} and the interim answer: {response}, determine if the interim answer satisfies the question.
            This does not need to be very strict. As long as the interim answer is able to address the question partially, return True."""
        )
        
        ## Initialize apple agent
        self.apple_agent = apple_agent
        
        ## Initialize DSPy definition service
        self.definition_service = HydeRAG()
        self.definition_service.load("../compiled.json")
    
    ## TODO: Method to prepare chat history ##
    
    ## TODO: Method to stuff chat history for dspy ##
    
    @step
    async def route(self, ctx: Context, ev: StartEvent) -> AppleEvent | InvestopediaEvent:
        selector_result = await self.router.aselect(self.choices, query = ev.query)
        await ctx.set("routes", 0)
        for result in selector_result.selections:
            if result.index == 0:
                self.send_event(AppleEvent(query=ev.query))
            else:
                self.send_event(InvestopediaEvent(query=ev.query))
            routes = await ctx.get("routes")
            routes += 1
            await ctx.set("routes", routes)
        
    @step
    async def apple_event(self, ev: AppleEvent) -> CompileEvent:
        response = await self.apple_agent.achat(ev.query, history=ev.history)
        return CompileEvent(query=ev.query, response = str(response))

    @step
    async def investopedia_event(self, ev: InvestopediaEvent) -> CompileEvent:
        response = self.definition_service(ev.query)
        return CompileEvent(query=ev.query, response = str(response))
    
    @step
    async def compile(self, ctx: Context, ev: CompileEvent) -> StopEvent:
        routes = ctx.get("routes")
        ready = ctx.collect_events(ev, [CompileEvent] * routes)
        for i in range(len(ready)):
            print(ready[i].source)
            reroute = await self.llm.structured_predict(
                Reroute, self.reroute_prompt_template, query=ev.query, response = ev.response
            )
            if reroute.reroute is True:
                if ready[i].source == "...":
                    return AppleEvent(query = f"""Here's some feedback to better address the query. 
                                      Feedback: {reroute.improvements},
                                      Query: {ev.query}""")
                else:
                    return InvestopediaEvent(query = f"""Here's some feedback to better address the query. 
                                            Feedback: {reroute.improvements},
                                            Query: {ev.query}""")
        response = await self.manager_llm.achat(
            f""""""
        )
        return StopEvent(result=ready[i].response)

# 2. Bringing Langgraph into the picture